In [1]:
!pip install transformers
!pip install hazm
!pip install faiss-gpu==1.6.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.3 MB/s 
     |████████████████████████████████| 120 kB 66.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 29.7 MB/s 
     |████████████████████████████████| 233 kB 69.8 MB/s 
     |████████████████████████████████| 1.4 MB 40.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=54e9945092776c30f3920a67870ea711f403da5e54bfb2fe071d90cc076d8add
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155070 sha256=7706098fd79c8ac267760f4258dafa61078e10c10c483b6823c8c3f823f6dbf0
  Stored in directory: /root/.cac

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_dir = "drive/MyDrive/UNI/CS/Term4/NLP/final_project/"

In [4]:
import codecs
import tqdm

mesra_collection = [x.strip().split() for x in tqdm.tqdm(codecs.open(F'{path_dir}/qazals/all_qazals_mesra.txt','rU','utf-8').readlines())]
train_mesras = mesra_collection[:20000]
test_mesras = mesra_collection[20000:20025]

100%|██████████| 329707/329707 [00:00<00:00, 397570.52it/s]


**Preprocessing**

In [5]:
from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()
lemmatizer = Lemmatizer()

mesra_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(train_mesras)]
mesra_sentences = [sent_tokenize(' '.join(x))[0] for x in tqdm.tqdm(mesra_normalized)]

test_mesra_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(test_mesras)]
test_mesra_sentences = [sent_tokenize(' '.join(x))[0] for x in tqdm.tqdm(test_mesra_normalized)]

100%|██████████| 25/25 [00:00<00:00, 68894.61it/s]


**Importing and Initializing the Model**

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AdamW,pipeline
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
model_inputs = tokenizer(mesra_sentences,
          add_special_tokens =True,
          padding='max_length',
          max_length=512,
          truncation=True,
          return_token_type_ids =False,
          return_tensors = 'pt',)
model_inputs.input_ids.shape

torch.Size([20000, 512])

In [8]:
import torch
model_inputs['labels'] = model_inputs.input_ids.detach().clone()
masks = (torch.rand(model_inputs.input_ids.shape) < .15) *\
        (model_inputs.input_ids != tokenizer.cls_token_id) *\
        (model_inputs.input_ids != tokenizer.sep_token_id) *\
        (model_inputs.input_ids != tokenizer.pad_token_id)
selection = []
for i in range(masks.shape[0]):
    selection.append(torch.flatten(masks[i].nonzero()).tolist())
#selection[0:5]

In [9]:
for i in range(masks.shape[0]):
    model_inputs.input_ids[i,selection[i]] = tokenizer.mask_token_id

In [10]:
class NewsDataset(torch.utils.data.Dataset):

    def __init__(self,encodings):
        self.encodings = encodings

    def __getitem__(self,idx):
        return {key : torch.tensor(val[idx]) for key,val in self.encodings.items()}
    
    def __len__(self): 
        return len(self.encodings['input_ids'])
dataset = NewsDataset(model_inputs)
train_dataloader = DataLoader(dataset , shuffle=True, batch_size= 4)

In [11]:
%%capture
model.train() #activating training mode
optimizer = AdamW(model.parameters(), lr=5e-5)

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

**Training the Model**

In [13]:
mode_="test"
path_dir = "/drive/MyDrive/UNI/CS/Term4/NLP/final_project/"

if mode_=="train":
    
    epochs = 3
    from tqdm import tqdm
    for epoch in range(epochs):
        loop = tqdm(train_dataloader, leave = True)
        for batch in loop:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids=input_ids,attention_mask = attention_mask,labels = labels)
            loss = output.loss
            loss.backward()
            optimizer.step()
            loop.set_description(f'Epoch:{epoch}')
            loop.set_postfix(loss = loss.item())

else:
    model.load_state_dict(torch.load("drive/MyDrive/UNI/CS/Term4/NLP/final_project/parsbert_finetuned_mesra"))

In [14]:
# torch.save(model.state_dict(), "drive/MyDrive/UNI/CS/Term4/NLP/final_project/parsbert_finetuned_mesra")

In [15]:
#for encoding we delete Mhead
model.cls= torch.nn.Identity()

In [43]:
def input_to_feature(input_,model):
    text_preprocessed = input_
    encoded = tokenizer.batch_encode_plus([text_preprocessed],max_length=50, padding='max_length', truncation=True)
    encoded = {key:torch.LongTensor(value) for key, value in encoded.items()}
    encoded['input_ids']=encoded['input_ids'].cuda()
    encoded['token_type_ids']=encoded['token_type_ids'].cuda()
    encoded['attention_mask']=encoded['attention_mask'].cuda()
    with torch.no_grad(): 
            outputs = model(**encoded)
    feature_outputs = outputs[0].mean(1)
    return feature_outputs


def training_set_feature_bank(model,Data):
        
    feature_bank=[]
 
    batch_size = 10  
    for idx in range(0, len(Data ), batch_size):
        batch = Data [idx : min(len( Data), idx+batch_size)]
        
        # encoded = tokenizer(batch)
        
        encoded = tokenizer.batch_encode_plus(batch,max_length=50, padding='max_length', truncation=True)
    
        encoded = {key:torch.LongTensor(value) for key, value in encoded.items()}
        encoded['input_ids']=encoded['input_ids'].cuda()
        encoded['token_type_ids']=encoded['token_type_ids'].cuda()
        encoded['attention_mask']=encoded['attention_mask'].cuda()

        with torch.no_grad():
            
            outputs = model(**encoded)
            
            feature_bank.append(outputs[0].mean(1))

    feature_bank_t=torch.cat(feature_bank)
    return feature_bank_t

# Testing the Model

**evaluate test sentences**

In [36]:
model.eval()
feature_bank = training_set_feature_bank(model, mesra_sentences)
test_feature_bank = training_set_feature_bank(model, test_mesra_sentences)

In [44]:
from sklearn.neighbors import NearestNeighbors

def find_most_similar(_test_sentences_embedding, _train_sentences_embedding, _number_of_neighbors):
  knn = NearestNeighbors(n_neighbors=_number_of_neighbors)
  knn.fit(_train_sentences_embedding)
  most_similar = knn.kneighbors(_test_sentences_embedding) 
  return most_similar

In [45]:
most_similar = find_most_similar(test_feature_bank.cpu(), feature_bank.cpu(), 10)

In [46]:
for main_sent_index, main_sent in enumerate(most_similar[1]):
  print("\n مصرع اصلی شماره " + str(main_sent_index) + ":‌ " + test_mesra_sentences[main_sent_index])
  for close_sent_index, close_sent in enumerate(main_sent):
    print(str(close_sent_index) + " : " + mesra_sentences[close_sent])


 مصرع اصلی شماره 0:‌ که نه امشب آن سماعست که دف خلاص یابد
0 : نه آن شبست که کس در میان ما گنجد
1 : هر که با شاهد گلروی به خلوت بنشست
2 : نشان عاشق آن باشد که شب با روز پیوندد
3 : که گر چه رنج به جان می‌رسد امید دواست
4 : که قیامت رسد این رشته به هم یا نرسد
5 : که می‌رود به شفاعت که دوست بازآرد
6 : که گفته‌ست که صد دل به غمزه‌ای ببری
7 : هر که با مستان نشیند ترک مستوری کند
8 : هر آن دل را که پنهانی قرینی هست روحانی
9 : که دل اهل نظر برد که سریست خدایی

 مصرع اصلی شماره 1:‌ به طپانچه‌ای و بربط برهد به گوشمالی
0 : به طمع ز دست رفتی و به پای درفکندت
1 : به قول مطرب و ساقی برون رفتم گه و بی‌گه
2 : سرو می‌نازد و خوش نیست خدا را بخرام
3 : دلق حافظ به چه ارزد به می‌اش رنگین کن
4 : صبا به غالیه سایی و گل به جلوه گری
5 : راکبی تندست و مرکوبی جمام
6 : که دست صبر برپیچید و بشکست
7 : ور نه بسیار بجویی و نیابی بازم
8 : بر لب جوی طرب جوی و به کف ساغر گیر
9 : تو مستریح و به افسوس می‌رود ایام

 مصرع اصلی شماره 2:‌ دگر آفتاب رویت منمای آسمان را
0 : چه پروای سخن گفتن بود مشتاق خدمت را
1 : ای دلیل دل گمگ

**Testing one sentence**

In [47]:
single_test_mesra_sentence = ["چراغ مرده شمع آفتاب"]
single_test_feature_bank = training_set_feature_bank(model, single_test_mesra_sentence)
single_test_most_similar = find_most_similar(single_test_feature_bank.cpu(), feature_bank.cpu(), 10)

In [48]:
for main_sent_index, main_sent in enumerate(single_test_most_similar[1]):
  print("\n مصرع اصلی شماره " + str(main_sent_index) + ":‌ " + single_test_mesra_sentence[main_sent_index])
  for close_sent_index, close_sent in enumerate(main_sent):
    print(str(close_sent_index) + " : " + mesra_sentences[close_sent])


 مصرع اصلی شماره 0:‌ چراغ مرده شمع آفتاب
0 : چراغ مرده کجا شمع آفتاب کجا
1 : شمع فلک با هزار مشعل انجم
2 : مه پیکر آفتاب پرتو
3 : آفتاب حسن او تا شعله زد
4 : چراغ دولتش نوری ندارد
5 : پرده برانداز شبی شمع وار
6 : چراغ صاعقه آن سحاب روشن باد
7 : نور ستارگان ستد روی چو آفتاب تو
8 : ببار ای شمع اشک از چشم خونین
9 : پیش بمیرد چراغدان ثریا
